## STEP 1:Load Raw Data

In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv('Dataset_Uber Traffic.csv')

# Preview
df.head()
df.info()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48120 entries, 0 to 48119
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DateTime  48120 non-null  object
 1   Junction  48120 non-null  int64 
 2   Vehicles  48120 non-null  int64 
 3   ID        48120 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


DateTime    0
Junction    0
Vehicles    0
ID          0
dtype: int64

## STEP 2: Clean the Data

In [10]:
# Drop duplicates
df = df.drop_duplicates()

In [12]:
# Check missing values
df.isnull().sum()

DateTime    0
Junction    0
Vehicles    0
ID          0
dtype: int64

In [14]:
# Fill missing values in 'Vehicles' column with median value
df['Vehicles'] = df['Vehicles'].fillna(df['Vehicles'].median())

In [18]:
df.dtypes


DateTime    datetime64[ns]
Junction             int64
Vehicles             int64
ID                   int64
dtype: object

In [16]:
# Convert DateTime column to proper datetime format
df['DateTime'] = pd.to_datetime(df['DateTime'])


## STEP 3: Aggregate traffic data
✅ STEP 1: Load & Prepare Time Columns

In [23]:
import pandas as pd

# Load the CSV
df = pd.read_csv("Dataset_Uber Traffic.csv")


In [27]:
# Convert to datetime
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%Y-%m-%d %H:%M:%S')


# Extract useful time components
df['Date'] = df['DateTime'].dt.date
df['Hour'] = df['DateTime'].dt.hour



✅ STEP 2: Group by Junction, Date, and Hour


In [30]:
# Group to get hourly traffic volume per junction
hourly_df = df.groupby(['Junction', 'Date', 'Hour'])['Vehicles'].sum().reset_index()


In [57]:
#Quick test to check formats:
print(df['DateTime'].head(10))

0   2015-11-01 00:00:00
1   2015-11-01 01:00:00
2   2015-11-01 02:00:00
3   2015-11-01 03:00:00
4   2015-11-01 04:00:00
5   2015-11-01 05:00:00
6   2015-11-01 06:00:00
7   2015-11-01 07:00:00
8   2015-11-01 08:00:00
9   2015-11-01 09:00:00
Name: DateTime, dtype: datetime64[ns]


## STEP 4: Preprocess the data

In [62]:
# I'll go with Standardization (z-score), as it's more robust for time series with outliers.
from sklearn.preprocessing import StandardScaler

# Assume you already have hourly_df from the last step
scaler = StandardScaler()

# Only scale the 'Vehicles' column
hourly_df['Vehicles_scaled'] = scaler.fit_transform(hourly_df[['Vehicles']])


In [68]:
# Aiming to view the scaled vehicle data in a clean tabular format
from tabulate import tabulate
print(tabulate(hourly_df[['Junction', 'Date', 'Hour', 'Vehicles', 'Vehicles_scaled']].head(), headers='keys', tablefmt='github'))


|    |   Junction | Date       |   Hour |   Vehicles |   Vehicles_scaled |
|----|------------|------------|--------|------------|-------------------|
|  0 |          1 | 2015-01-11 |      0 |         15 |         -0.375489 |
|  1 |          1 | 2015-01-11 |      1 |         13 |         -0.471875 |
|  2 |          1 | 2015-01-11 |      2 |         10 |         -0.616454 |
|  3 |          1 | 2015-01-11 |      3 |          7 |         -0.761034 |
|  4 |          1 | 2015-01-11 |      4 |          9 |         -0.664648 |


In [72]:
# save the Preprocessed Data
hourly_df.to_csv("Preprocessed_Traffic_Data.csv", index=False)


## Feature Engineering and Selection

✅ STEP 1: Create New Time-Based Features

In [80]:
hourly_df['DateTime'] = pd.to_datetime(hourly_df['Date'].astype(str) + ' ' + hourly_df['Hour'].astype(str) + ':00:00')


In [82]:
# Convert DateTime if needed
hourly_df['DateTime'] = pd.to_datetime(hourly_df['DateTime'])
# Extract time-based features
hourly_df['Hour'] = hourly_df['DateTime'].dt.hour
hourly_df['DayOfWeek'] = hourly_df['DateTime'].dt.dayofweek  # Monday = 0
hourly_df['Month'] = hourly_df['DateTime'].dt.month

# Weekend binary feature
hourly_df['IsWeekend'] = hourly_df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)


In [84]:
print(hourly_df.columns)


Index(['Junction', 'Date', 'Hour', 'Vehicles', 'Vehicles_scaled', 'DateTime',
       'DayOfWeek', 'Month', 'IsWeekend'],
      dtype='object')


✅ STEP 2: Create Lag Features

In [93]:
# Lag traffic count by 1 and 2 hours
hourly_df['Lag_1'] = hourly_df.groupby('Junction')['Vehicles'].shift(1)
hourly_df['Lag_2'] = hourly_df.groupby('Junction')['Vehicles'].shift(2)


✅ STEP 3: Add Special Event Indicator

In [95]:
# Placeholder: Assume no special events for now
hourly_df['IsEvent'] = 0


✅ STEP 4: Evaluate Feature Importance

In [99]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Drop NaN rows (from lag features)
hourly_df = hourly_df.dropna()

# Define features and target
features = ['Hour', 'DayOfWeek', 'Month', 'IsWeekend', 'Lag_1', 'Lag_2', 'IsEvent']
X = hourly_df[features]
y = hourly_df['Vehicles']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit Random Forest
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

# Feature importances
importances = rf.feature_importances_
for feat, imp in zip(features, importances):
    print(f"{feat}: {imp:.4f}")


Hour: 0.0238
DayOfWeek: 0.0065
Month: 0.0095
IsWeekend: 0.0011
Lag_1: 0.9437
Lag_2: 0.0154
IsEvent: 0.0000


✅ STEP 5: Final Dataset for Modeling

In [102]:
hourly_df.to_csv("Feature_Engineered_Traffic_Data.csv", index=False)


## Industry Research: Impact of Traffic on Uber Fare Pricing

#### Traffic significantly influences Uber's fare pricing due to dynamic pricing models like surge pricing. During peak hours, high demand and low driver availability lead to increased fares. Traffic congestion increases trip duration, which directly raises time-based charges. Events, public holidays, and bad weather also lead to traffic spikes, causing longer wait times and route detours — all of which raise fare estimates. This not only impacts customer cost but also affects Uber’s supply-demand balance and driver earnings.

## Data Collection and Integration

In [115]:
#inspecting the actual column names
print(weather_merged.columns)


Index(['name', 'datetime', 'tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
       'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations'],
      dtype='object')


## 🔹 WEATHER DATA:

In [150]:
import pandas as pd

# Load both CSV files
weather_1 = pd.read_csv('Delhi, India 2015-01-11 to 2016-06-30.csv')
weather_2 = pd.read_csv('delhi 2016-07-01 to 2017-12-31.csv')

# Merge (concatenate) both datasets
weather_merged = pd.concat([weather_1, weather_2], ignore_index=True)

# Convert date column to datetime
weather_merged['datetime'] = pd.to_datetime(weather_merged['datetime'])

# Sort by date just in case
weather_merged = weather_merged.sort_values('datetime')

# Check output
weather_merged.head()


,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"Delhi, India",2015-01-11,59.0,44.6,50.5,59.0,44.2,49.9,45.9,85.5,...,12.1,5,NaN,2015-01-11T07:15:19,2015-01-11T17:42:35,0.70,Partially cloudy,Partly cloudy throughout the day.,fog,"42182099999,VIDP,42181099999"
1,"Delhi, India",2015-01-12,60.8,44.1,51.4,60.8,43.5,50.9,47.1,86.4,...,12.2,6,NaN,2015-01-12T07:15:18,2015-01-12T17:43:23,0.73,Partially cloudy,Partly cloudy throughout the day.,fog,"42182099999,VIDP,42181099999,remote"
2,"Delhi, India",2015-01-13,63.4,42.8,51.6,63.4,41.1,50.9,47.4,86.8,...,12.0,6,NaN,2015-01-13T07:15:16,2015-01-13T17:44:11,0.75,Partially cloudy,Partly cloudy throughout the day.,fog,"42182099999,VIDP,42181099999,remote"
3,"Delhi, India",2015-01-14,59.0,48.8,53.9,59.0,48.8,53.9,49.9,86.6,...,10.6,5,NaN,2015-01-14T07:15:13,2015-01-14T17:45:00,0.80,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"42182099999,VIDP,42181099999"
4,"Delhi, India",2015-01-15,64.4,41.0,51.9,64.4,40.0,51.5,47.0,85.4,...,13.7,6,NaN,2015-01-15T07:15:08,2015-01-15T17:45:49,0.83,Partially cloudy,Clearing in the afternoon.,fog,"42182099999,VIDP,42181099999,remote"


## STEP 3: Add EVENT DATA

manually creating a DataFrame for key event dates

In [153]:
# Manually define public holidays and events in Delhi
events = {
    'DateTime': [
        '2015-08-15 00:00:00',  # Independence Day
        '2015-10-02 00:00:00',  # Gandhi Jayanti
        '2016-01-26 00:00:00',  # Republic Day
        '2016-08-15 00:00:00',  # Independence Day
        '2016-10-30 00:00:00',  # Diwali
        '2017-08-15 00:00:00',  # Independence Day
        '2017-10-02 00:00:00',  # Gandhi Jayanti
    ],
    'IsEvent': [1]*7  # Binary indicator
}

event_df = pd.DataFrame(events)
event_df['DateTime'] = pd.to_datetime(event_df['DateTime'])


## STEP 4: Merge All Data

In [156]:
# 🔗 Merge traffic and weather datasets on their datetime columns

# 'DateTime' from traffic and 'datetime' from weather_merged
merged = pd.merge(traffic, weather_merged, left_on='DateTime', right_on='datetime', how='left')

# 📅 Merge the result with event data to add public holiday/event indicators
# This uses 'DateTime' as the common key
final_merged = pd.merge(merged, event_df, on='DateTime', how='left')

# 🧹 Fill missing event indicators with 0 (i.e., non-event days)
final_merged['IsEvent'] = final_merged['IsEvent'].fillna(0)



## STEP 5: Handle Data Quality Issues

In [159]:
# 📉 Check how many missing values exist in each column of the final merged dataset
missing_summary = final_merged.isnull().sum()

# 🧹 Remove any duplicate rows to ensure data integrity
final_merged = final_merged.drop_duplicates()

# 🔄 Fill missing values using forward fill (ffill)
# This method carries forward the last known value, useful for weather data continuity
final_merged = final_merged.ffill()


# 📐 Normalize selected numeric features to a 0–1 range using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# 🔧 Specify which columns to scale (adjust based on your actual column names)
cols_to_scale = ['temp', 'humidity', 'windspeed', 'Vehicles']  # Example

# 🧪 Apply scaling to the selected columns
final_merged[cols_to_scale] = scaler.fit_transform(final_merged[cols_to_scale])


In [161]:
print(final_merged.columns.tolist())


['DateTime', 'Junction', 'Vehicles', 'ID', 'name', 'datetime', 'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise', 'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations', 'IsEvent']


In [164]:
# Saving Integrated Dataset
final_merged.to_csv('Final_Integrated_Dataset.csv', index=False)
